In [1]:
import pandas as pd 
from sqlalchemy import create_engine

#### Read product data from stg_product

#### Create a connection to SQL Srever 

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

#### Read Product Data from stg_product table

In [3]:
sql_stg_product = '''SELECT *
                  FROM  stg_product'''

In [4]:
# store data to a dataframe 
stg_product_df = pd.read_sql(sql_stg_product,engine)
stg_product_df.head()

,product_code,product_line
0,S10_1678,Motorcycles
1,S10_1678,Motorcycles
2,S10_1678,Motorcycles
3,S10_1678,Motorcycles
4,S10_1678,Motorcycles


In [5]:
len(stg_product_df)

2823

#### Cleand Product Data 

In [6]:
# check distinict product_code values 
stg_product_df['product_code'].value_counts()

product_code
S18_3232    52
S12_1666    28
S10_4962    28
S18_1097    28
S10_1949    28
            ..
S18_2248    22
S24_2887    22
S18_4933    22
S18_4409    22
S24_3969    22
Name: count, Length: 109, dtype: int64

In [7]:
# Check for null values 
stg_product_df['product_code'].isnull().sum()


np.int64(0)

##### Drop duplicates

In [8]:
# Drop duplicates
stg_product_df= stg_product_df.drop_duplicates()

In [9]:
len(stg_product_df)

109

In [10]:
stg_product_df.head(10)

,product_code,product_line
0,S10_1678,Motorcycles
26,S10_1949,Classic Cars
54,S10_2016,Motorcycles
80,S10_4698,Motorcycles
106,S10_4757,Classic Cars
133,S10_4962,Classic Cars
161,S12_1099,Classic Cars
186,S12_1108,Classic Cars
212,S12_1666,Trucks and Buses
240,S12_2823,Motorcycles


In [11]:
stg_product_df = stg_product_df.copy()
stg_product_df = stg_product_df.reset_index(drop = True)
stg_product_df

,product_code,product_line
0,S10_1678,Motorcycles
1,S10_1949,Classic Cars
2,S10_2016,Motorcycles
3,S10_4698,Motorcycles
4,S10_4757,Classic Cars
...,...,...
104,S700_3505,Ships
105,S700_3962,Ships
106,S700_4002,Planes
107,S72_1253,Planes


#### Load Data to DimProduct Using Incremental Load Approach

##### 1) Load Product Data from DimProduct Table

In [12]:
# load DimProduct table into a DataFrame
sql_DimProduct = '''SELECT *
                  FROM  DimProduct'''

In [13]:
dim_product_df = pd.read_sql(sql_DimProduct, engine)
dim_product_df.head(7)

,product_code,product_line


##### 2) Perform stg_product left join DimProduct tables to find new records 

In [14]:
# Merge on a key column 
merged_product_df = stg_product_df.merge(dim_product_df, on = ['product_code']
                                 , how = 'left', indicator = True, suffixes = ('','_dim'))
merged_product_df.head()

,product_code,product_line,product_line_dim,_merge
0,S10_1678,Motorcycles,NaN,left_only
1,S10_1949,Classic Cars,NaN,left_only
2,S10_2016,Motorcycles,NaN,left_only
3,S10_4698,Motorcycles,NaN,left_only
4,S10_4757,Classic Cars,NaN,left_only


In [15]:
# Filter records that are only in staging but not in DimProduct
new_product_records_df = merged_product_df[merged_product_df['_merge']== 'left_only']
new_product_records_df.head()

,product_code,product_line,product_line_dim,_merge
0,S10_1678,Motorcycles,NaN,left_only
1,S10_1949,Classic Cars,NaN,left_only
2,S10_2016,Motorcycles,NaN,left_only
3,S10_4698,Motorcycles,NaN,left_only
4,S10_4757,Classic Cars,NaN,left_only


In [16]:
# drop columns from DimProduct and keep only staging data
new_product_records_df= new_product_records_df[stg_product_df.columns]
new_product_records_df.head()

,product_code,product_line
0,S10_1678,Motorcycles
1,S10_1949,Classic Cars
2,S10_2016,Motorcycles
3,S10_4698,Motorcycles
4,S10_4757,Classic Cars


##### 3) Insert new records into DimProduct  MS Server SQL Table 

In [17]:
# insert new records to the target table
total_inserted_product_records = new_product_records_df.to_sql('DimProduct', con = engine, if_exists = 'append', index = False)
print(total_inserted_product_records)

109
